In [ ]:
from capella import lee_filter
import numpy as np
from scipy.ndimage.filters import uniform_filter, minimum_filter, maximum_filter, median_filter
from scipy.ndimage.measurements import variance
from matplotlib import pyplot as plt 
from skimage import io
from skimage.feature import peak_local_max, match_template
from skimage.color import rgb2gray
import math

In [ ]:
def stretch(bands, lower_percent=2, higher_percent=98):
    np.ma.array(bands, mask=np.isnan(bands))
    out = np.zeros_like(bands)
    a = 0 
    b = 255 
    c = np.percentile(bands, lower_percent)
    d = np.percentile(bands, higher_percent)        
    t = a + (bands - c) * (b - a) / (d - c)    
    t[t<a] = a
    t[t>b] = b
    out = t
    return out.astype(np.uint8)

In [ ]:
def check_grayscale(img):
    if len(img.shape) > 2:
        img_gry = rgb2gray(img)
    else:
        img_gry = img
        
    return img_gry 

In [ ]:
image = io.imread('test_sar_band_1.jpg')
template = io.imread('test_sar_band_1_template.jpg')

# Convert to grayscale
imageGray = check_grayscale(image)
templateGray = check_grayscale(template)
w, h = templateGray.shape[::-1]

# despeckle images
filtsize = 3
imageGray = lee_filter(imageGray, filtsize).astype(np.float32) 
templateGray = lee_filter(templateGray, filtsize).astype(np.float32) 

result = np.squeeze(match_template(imageGray, templateGray))

peaks = peak_local_max(result, threshold_abs=0.5, min_distance=3)

fig = plt.figure(figsize=(15, 15))
ax1 = plt.subplot(1, 3, 1)
ax2 = plt.subplot(1, 3, 2)
ax3 = plt.subplot(1, 3, 3, sharex=ax2, sharey=ax2)

ax1.imshow(templateGray, cmap=plt.cm.gray)
ax1.set_axis_off()
ax1.set_title('template')

ax2.imshow(imageGray, cmap=plt.cm.gray)
ax2.set_axis_off()
ax2.set_title('image')

# highlight matched regions
h_match, w_match = templateGray.shape
for pk in peaks:
    rect = plt.Rectangle((pk[1], pk[0]), w_match, h_match, edgecolor='r', facecolor='none')
    ax2.add_patch(rect)
    ax3.plot(pk[1], pk[0], 'o', markeredgecolor='r', markerfacecolor='none', markersize=10)

ax3.imshow(result)
ax3.set_axis_off()
ax3.set_title('`match template`\nresult')
ax3.autoscale(False)

plt.show()

print(f"Matched {len(peaks)}")
